In [ ]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
# from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
# from keras.layers.normalization import BatchNormalization
from tensorflow.keras.layers import BatchNormalization
import os
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv("/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/train.csv")
# train = pd.read_csv('train.csv')
test = pd.read_csv("/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/test.csv")

# test = pd.read_csv('test.csv')

train['image_id'] = train['image_id']+'.jpg'
test['image_id'] = test['image_id'] +'.jpg'

train.head()

,image_id,healthy,multiple_diseases,rust,scab
0,Train_0.jpg,0,0,0,1
1,Train_1.jpg,0,1,0,0
2,Train_2.jpg,1,0,0,0
3,Train_3.jpg,0,0,1,0
4,Train_4.jpg,1,0,0,0


In [ ]:
train.shape

(1821, 5)

In [ ]:
img=[]
filename=train.image_id
for file in filename:
    image=cv2.imread("/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/images/"+file)
    res=cv2.resize(image,(256,256))
    img.append(res)
img=np.array(img)

In [ ]:
img.shape

(1821, 256, 256, 3)

In [ ]:
train_labels = np.float32(train.loc[:, 'healthy':'scab'].values)

In [ ]:
train_labels.shape

(1821, 4)

In [ ]:
train, val = train_test_split(train, test_size = 0.15)

In [ ]:
print(train.shape)

(1547, 5)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator( 
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=.1,
    fill_mode='nearest',
    shear_range=0.1,
    rescale=1/255,
    brightness_range=[0.5, 1.5])

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
                    train, directory = '/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/images/',

                    target_size = (128,128),
                    x_col = 'image_id',
                    y_col = ['healthy','multiple_diseases','rust','scab'],
                    class_mode='raw',
                    shuffle=False,
                    subset='training',
                    batch_size = 16
)

Found 1547 validated image filenames.


In [ ]:
val_generator=train_datagen.flow_from_dataframe(val,directory='/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/images/',
                                                      target_size=(128,128),
                                                      x_col="image_id",
                                                      y_col=['healthy','multiple_diseases','rust','scab'],
                                                      class_mode='raw',
                                                      shuffle=False,
                                                      batch_size=16,
                                                  )

Found 274 validated image filenames.


In [ ]:
test_generator=train_datagen.flow_from_dataframe(test,directory='/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/images/',
                                                      target_size=(128,128),
                                                      x_col="image_id",
                                                      y_col=None,
                                                      class_mode=None,
                                                      shuffle=False,
                                                      batch_size=16)

Found 1821 validated image filenames.


In [ ]:
from keras.applications.densenet import DenseNet121
from keras.models import Model
import keras
from keras import optimizers

import tensorflow as tf

In [ ]:
# imagenet --> none
pretrained_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(128,128,3))
# include_top = True -- full connected neural network

In [ ]:
model = tf.keras.Sequential([
    pretrained_model,
    # --> maxpooling2D
    tf.keras.layers.GlobalAveragePooling2D(),
    #  --> droout --> [0.3- 0.5]
    tf.keras.layers.Dropout(0.3),
    # thêm layer vào đây nếu thích hợp
    tf.keras.layers.Dense(4, activation='softmax')  
    ])

In [ ]:
model.compile(
    # thay Adam --> vài hàm khác để chạy thử như thế
    optimizer=tf.keras.optimizers.Adam(),
    loss= tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['acc']
)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 densenet121 (Functional)    (None, 4, 4, 1024)        7037504   
                                                                 
 global_average_pooling2d (G  (None, 1024)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 4)                 4100      
                                                                 
Total params: 7,041,604
Trainable params: 6,957,956
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
from keras.callbacks import ReduceLROnPlateau

In [ ]:
history_1 = model.fit_generator(train_generator,
                               steps_per_epoch=10,
                                # thay epochs tăng từ từ lên tầm 250
                               epochs=150, validation_data=val_generator,
                               use_multiprocessing=False,
                               shuffle=True
                               )

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/150


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


10/10 [==============================] - 40s 2s/step - loss: 1.7537 - acc: 0.4750 - val_loss: 3.0246 - val_acc: 0.3650
Epoch 2/150
10/10 [==============================] - 17s 2s/step - loss: 1.0237 - acc: 0.6625 - val_loss: 117.7818 - val_acc: 0.2701
Epoch 3/150
10/10 [==============================] - 17s 2s/step - loss: 1.0094 - acc: 0.7000 - val_loss: 411.9474 - val_acc: 0.2701
Epoch 4/150
10/10 [==============================] - 17s 2s/step - loss: 0.7401 - acc: 0.7375 - val_loss: 897.4506 - val_acc: 0.2701
Epoch 5/150
10/10 [==============================] - 17s 2s/step - loss: 0.7377 - acc: 0.7500 - val_loss: 256.1422 - val_acc: 0.2701
Epoch 6/150
10/10 [==============================] - 17s 2s/step - loss: 0.8478 - acc: 0.7563 - val_loss: 91.0972 - val_acc: 0.3029
Epoch 7/150
10/10 [==============================] - 17s 2s/step - loss: 0.6295 - acc: 0.7625 - val_loss: 89.1097 - val_acc: 0.3321
Epoch 8/150
10/10 [==============================] - 17s 2s/step - loss: 0.6483 - acc

In [ ]:
SUB_PATH = "/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/sample_submission.csv"

sub = pd.read_csv(SUB_PATH)
probs_RESNET = model.predict(test_generator, verbose=1)
sub.loc[:, 'healthy':] = probs_RESNET
sub.to_csv('/content/drive/MyDrive/NhanDang/NeuralNetwork/Plant-Pathology-2020---FGVC7-master/submission_test.csv', index=False)
sub.head()

114/114 [==============================] - 81s 710ms/step


,image_id,healthy,multiple_diseases,rust,scab
0,Test_0,1.437271e-03,5.768953e-02,9.249712e-01,1.590202e-02
1,Test_1,1.299925e-04,3.813896e-03,9.959292e-01,1.268632e-04
2,Test_2,9.294681e-06,1.647142e-04,5.282595e-07,9.998255e-01
3,Test_3,9.986542e-01,1.844354e-04,2.116230e-04,9.497402e-04
4,Test_4,3.153705e-12,4.119640e-07,9.999996e-01,5.644406e-11


In [ ]:
tf.keras.models.save_model(model, 'my_model.hdf5')